In [41]:
from nltk.corpus import movie_reviews
import nltk
import random
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
import string

In [32]:
movie_reviews.categories()
lemmatizer=WordNetLemmatizer()

In [47]:
stops=stopwords.words('english')
punc=list(string.punctuation)
stops=stops+punc

In [15]:
len(movie_reviews.fileids())
#1000 -ve
#1000 +ve       

2000

In [23]:
#it returns file id with positive reviews
pos_rev=movie_reviews.fileids('pos')

In [24]:
#we need to pass the file id here
movie_reviews.words(movie_reviews.fileids())

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [26]:
documents=[]
for category in movie_reviews.categories():
    for fid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fid),category))
documents[1]        

(['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg')

***Shuffling to mix pos/neg so that it becomes easy for train_test_split***

In [28]:
random.shuffle(documents)

In [36]:
def getsimplepos (tag) :
    if tag.startswith( 'J'):
        return wordnet.ADJ
    elif tag.startswith( 'V'):
        return wordnet. VERB
    elif tag.startswith( 'N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet .NOUN

**pos_tag requires an array**

In [39]:
w="better"
pos_tag([w])

[('better', 'RBR')]

**Cleaning the dataset**

In [40]:
def clean_review(words):
    output_words=[]
    for w in words:
        if(w.lower() not in stops):
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=getsimplepos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words        
            

In [48]:
documents=[(clean_review(document),category) for document,category in documents]

In [50]:
documents[0]

(['michael',
  'crichton',
  'long',
  'career',
  'write',
  'novel',
  'many',
  'science',
  'fiction',
  'profitable',
  'film',
  'adaptation',
  'novel',
  'adaptation',
  'michael',
  'crichton',
  'science',
  'fiction',
  'novel',
  'logic',
  'film',
  'industry',
  'good',
  'way',
  'make',
  'profitable',
  'film',
  'would',
  'make',
  'big',
  'budget',
  'adaptation',
  'another',
  'crichton',
  'science',
  'fiction',
  'novel',
  'congo',
  'fail',
  'afraid',
  'sphere',
  'probably',
  'go',
  'fare',
  'whole',
  'lot',
  'well',
  'little',
  'well',
  'okay',
  'novel',
  'make',
  'film',
  'even',
  'good',
  'film',
  'expensive',
  'one',
  'hundred',
  'million',
  'dollar',
  'long',
  '133',
  'minute',
  'terrific',
  'cast',
  'include',
  'dustin',
  'hoffman',
  'samuel',
  'l',
  'jackson',
  'sharon',
  'stone',
  'little',
  'really',
  'original',
  'less',
  'excite',
  'several',
  'year',
  'ago',
  'dr',
  'norman',
  'goodman',
  'played',
 

**Now creating dataset with feature as top words**

***Building the feature sets***